### import

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
import datetime
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

## functions

In [ ]:
def str_2_float_nan(list_a):
  offer_price_list = []
  for price in [str(price).replace(",", "") for price in list_a]:
    try:
      tempt_1 = int(price)
      offer_price_list.append(str(tempt_1))
    except:
      tempt_2 = None
      offer_price_list.append(tempt_2)
  return offer_price_list

# Data

In [ ]:
# crsp data
crsp_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/unuysf43vpet2b1bpw8uu0m1bv54gguw.csv')
# create NCUSIP6 variable, equal to the first six characters of the NCUSIP variable
crsp_1995_2022['NCUSIP6'] =  crsp_1995_2022['NCUSIP'].astype(str).str[:6]
crsp_1995_2022.shape

<ipython-input-4-d862cf7cda72>:2: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/unuysf43vpet2b1bpw8uu0m1bv54gguw.csv')


(40917860, 10)

In [ ]:
crsp_1995_2022

,PERMNO,date,SHRCD,EXCHCD,NCUSIP,PRC,RET,SHROUT,OPENPRC,NCUSIP6
0,10001,1995-01-03,11,3,29274A10,8.375,0.046875,2224,8.500,29274A
1,10001,1995-01-04,11,3,29274A10,8.000,-0.044776,2224,8.000,29274A
2,10001,1995-01-05,11,3,29274A10,7.500,-0.062500,2224,8.000,29274A
3,10001,1995-01-06,11,3,29274A10,-7.750,0.033333,2224,NaN,29274A
4,10001,1995-01-09,11,3,29274A10,7.500,-0.032258,2224,7.500,29274A
...,...,...,...,...,...,...,...,...,...,...
40917855,93436,2022-12-23,11,3,88160R10,123.150,-0.017551,3157752,126.370,88160R
40917856,93436,2022-12-27,11,3,88160R10,109.100,-0.114089,3157752,117.495,88160R
40917857,93436,2022-12-28,11,3,88160R10,112.710,0.033089,3157752,110.350,88160R
40917858,93436,2022-12-29,11,3,88160R10,121.820,0.080827,3157752,120.385,88160R


In [ ]:
# sdc data
final_sdc_include_vd = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/final_sdc_df/final_sdc_include_vd.csv')
final_sdc_include_vd.drop('Unnamed: 0',axis=1,inplace=True)
final_sdc_include_vd

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed,over_subscription_flag_use,udwr_lead_aff_by_deal,udwr_any_aff_by_deal
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,8.663020,Within range,5.0,5.0,5.0,5.0,No,1,0,0
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,13.341051,Within range,7.0,9.0,7.0,9.0,No,1,0,0
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,9.356062,Within range,4.5,5.5,4.5,5.5,No,1,0,0
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,13.514311,Within range,5.0,5.0,5.0,5.0,Yes,1,0,0
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,8.663020,Within range,5.0,6.0,5.0,6.0,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5604,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,94.400000,Below range,16.0,18.0,16.0,18.0,No,1,0,0
5605,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,20.100000,Within range,4.5,5.0,4.5,5.0,No,1,0,0
5606,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,11.000000,Within range,4.5,5.0,4.5,5.0,No,1,0,0
5607,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,16.800000,Within range,5.0,7.0,5.0,7.0,No,1,0,0


In [ ]:
final_sdc_include_vd.columns

Index(['jkl_index', 'deal_number', 'filing_date', 'issue_date', 'issuer',
       'main_sic_code', 'nation', 'lead_underwriter_code', 'all_managers',
       'all_mgr_role_code', 'lead_underwriter', 'cusip', 'cusip9',
       'lead_underwriter_list', 'JB_JL_jointlead', 'CM_CL_comanager',
       'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
       'selling_conces_sion_($)', 're_allow_ance_fee_($)',
       'gross_spread_in_this_mkt_($_mil)', 'principal_amount_($_mil)',
       'proceeds_amt_in_this_mkt_($_mil)', 'offer_price', 'type_of_security',
       'description', 'marketplace',
       'primary_exchange_where_issue_will_be_listed', 'HFILE', 'LFILE',
       'original_middle_of_filing_price_range', 'shares_filed_in_this_mkt',
       'amt_filed_in_this_mkt_($_mil)', 'reit_type_code', 'depos_itary',
       'unit_inv._trust', 'closed_end_fund/trust_flag_(y/n)',
       'gross_spread_as_%_of_prncpl_amt_in_this_mkt', 'AH_LFILE', 'AH_HFILE',
       'change_in_value_of_shares

In [ ]:
# compustat sic code data
sic_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/kpqim4mj7j4a174khqfxrfo4yan3qax8.csv')
sic_1995_2022

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,cusip,conm,curcd,act,costat,mkvalt,sic
0,1004,1995-05-31,1994.0,INDL,C,D,STD,000361105,AAR CORP,USD,321.632,A,NaN,5080
1,1004,1996-05-31,1995.0,INDL,C,D,STD,000361105,AAR CORP,USD,338.012,A,NaN,5080
2,1004,1997-05-31,1996.0,INDL,C,D,STD,000361105,AAR CORP,USD,414.100,A,NaN,5080
3,1004,1998-05-31,1997.0,INDL,C,D,STD,000361105,AAR CORP,USD,468.400,A,NaN,5080
4,1004,1999-05-31,1998.0,INDL,C,D,STD,000361105,AAR CORP,USD,508.186,A,540.7748,5080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358061,352262,2022-12-31,2022.0,INDL,C,D,STD,G2415A113,COOL COMPANY LTD,USD,145.338,A,NaN,4400
358062,353444,2021-12-31,2021.0,INDL,C,D,STD,405552100,HALEON PLC,USD,7110.379,A,NaN,2834
358063,353444,2022-12-31,2022.0,INDL,C,D,STD,405552100,HALEON PLC,USD,4890.689,A,NaN,2834
358064,353945,2021-12-31,2021.0,INDL,C,D,STD,00449R109,ACCELLERON INDUSTRIES AG,USD,436.863,A,NaN,3621


In [ ]:
final_sdc_vd_sic_ib_fix2 = pd.read_csv('https://utexas.box.com/shared/static/99rz5iayzgkczmdbf5ppnfuyx1ph4hrc.csv')
final_sdc_vd_sic_ib_fix2.drop('Unnamed: 0',axis=1,inplace=True)
final_sdc_vd_sic_ib_fix2.reset_index(drop=True,inplace=True)
final_sdc_vd_sic_ib_fix2.shape

<ipython-input-7-0b02577087f1>:1: DtypeWarning: Columns (30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  final_sdc_vd_sic_ib_fix2 = pd.read_csv('https://utexas.box.com/shared/static/99rz5iayzgkczmdbf5ppnfuyx1ph4hrc.csv')


(5602, 199)

In [ ]:
final_sdc_vd_sic_ib_fix2

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,IB_fixed_legg mason wood walker,IB_fixed_national us,IB_fixed_adams harkness hill,IB_fixed_simmons energy,IB_fixed_bancamerica robertson stephen,IB_fixed_aegis capital,IB_fixed_furman selz,IB_fixed_jc bradford,IB_fixed_wessels arnold henderson ll,IB_fixed_pacific growth equities
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5597,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5598,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5599,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5600,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
final_sdc_vd_sic_ib_fix2.columns[:100]

In [ ]:
# unique permno list
CRSP_permno_list = list(crsp_1995_2022.PERMNO.unique())

In [ ]:
# sort values
crsp_1995_2022 = crsp_1995_2022.sort_values(['PERMNO', 'date'])

In [ ]:
crsp_1995_2022_dict = dict()
for permno in tqdm(CRSP_permno_list):
  crsp_1995_2022_dict[permno] = crsp_1995_2022[crsp_1995_2022.PERMNO == permno ]

100%|██████████| 19335/19335 [08:50<00:00, 36.43it/s]


In [ ]:
crsp_1995_2022_dict[permno]

In [ ]:
# save dictionary to crsp_1995_2022_dict.pkl file
with open('/content/drive/MyDrive/UT_research/VC_IB/code2/data/crsp_1995_2022_dict.pkl', 'wb') as fp:
    pickle.dump(crsp_1995_2022_dict, fp)

# Data 2

In [ ]:
# Read dictionary pkl file
with open('/content/drive/MyDrive/UT_research/VC_IB/code2/data/crsp_1995_2022_dict.pkl', 'rb') as fp:
    pk1 = pickle.load(fp)
    print('Person dictionary')

Person dictionary


In [ ]:
pk1.keys()

In [ ]:
pk1[84587].date.iloc[0]

In [ ]:
final_sdc_vd_sic_ib_fix2 = pd.read_csv('https://utexas.box.com/shared/static/99rz5iayzgkczmdbf5ppnfuyx1ph4hrc.csv')
final_sdc_vd_sic_ib_fix2.drop('Unnamed: 0',axis=1,inplace=True)
final_sdc_vd_sic_ib_fix2.reset_index(drop=True,inplace=True)
final_sdc_vd_sic_ib_fix2.shape

<ipython-input-10-0b02577087f1>:1: DtypeWarning: Columns (30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  final_sdc_vd_sic_ib_fix2 = pd.read_csv('https://utexas.box.com/shared/static/99rz5iayzgkczmdbf5ppnfuyx1ph4hrc.csv')


(5602, 199)

In [ ]:
# sdc data
final_sdc_include_vd = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/final_sdc_df/final_sdc_include_vd.csv')
final_sdc_include_vd.drop('Unnamed: 0',axis=1,inplace=True)
final_sdc_include_vd

In [ ]:
final_sdc_include_vd.columns[:100]

Index(['jkl_index', 'deal_number', 'filing_date', 'issue_date', 'issuer',
       'main_sic_code', 'nation', 'lead_underwriter_code', 'all_managers',
       'all_mgr_role_code', 'lead_underwriter', 'cusip', 'cusip9',
       'lead_underwriter_list', 'JB_JL_jointlead', 'CM_CL_comanager',
       'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
       'selling_conces_sion_($)', 're_allow_ance_fee_($)',
       'gross_spread_in_this_mkt_($_mil)', 'principal_amount_($_mil)',
       'proceeds_amt_in_this_mkt_($_mil)', 'offer_price', 'type_of_security',
       'description', 'marketplace',
       'primary_exchange_where_issue_will_be_listed', 'HFILE', 'LFILE',
       'original_middle_of_filing_price_range', 'shares_filed_in_this_mkt',
       'amt_filed_in_this_mkt_($_mil)', 'reit_type_code', 'depos_itary',
       'unit_inv._trust', 'closed_end_fund/trust_flag_(y/n)',
       'gross_spread_as_%_of_prncpl_amt_in_this_mkt', 'AH_LFILE', 'AH_HFILE',
       'change_in_value_of_shares

In [ ]:
final_sdc_vd_sic_ib_fix2.columns[:100]

In [ ]:
drop_list1= ['first_PRC', 'first_open_prc', 'PRC_7d',
            'PRC_1m', 'PRC_3m', 'PRC_6m', 'PRC_6m_2', 'PRC_6m_3', 'PRC_1y',
            'PRC_3y', 'PRC_5y', 'return_1w', 'return_1m', 'return_3m', 'return_6m',
            'return_6m_2', 'return_6m_3', 'return_1y', 'return_3y', 'return_5y',
            'pe_vc_backed', 'sic_return_initial', 'sic_return_1w',
            'sic_return_1m_31', 'sic_return_3m_94', 'sic_return_6m_184',
            'sic_return_6m_185', 'sic_return_1y', 'sic_return_5y']

In [ ]:
# our
final_sdc_vd_sic_ib_fix3 = final_sdc_vd_sic_ib_fix2.drop(drop_list1, axis=1)
final_sdc_vd_sic_ib_fix3

In [ ]:
final_sdc_vd_sic_ib_fix3.columns[:71]

Index(['jkl_index', 'deal_number', 'filing_date', 'issue_date', 'issuer',
       'main_sic_code', 'nation', 'lead_underwriter_code', 'all_managers',
       'all_mgr_role_code', 'lead_underwriter', 'cusip', 'cusip9',
       'lead_underwriter_list', 'JB_JL_jointlead', 'CM_CL_comanager',
       'gross_spread_($)', 'manage_ment_fee_($)', 'under_writing_fee_($)',
       'selling_conces_sion_($)', 're_allow_ance_fee_($)',
       'gross_spread_in_this_mkt_($_mil)', 'principal_amount_($_mil)',
       'proceeds_amt_in_this_mkt_($_mil)', 'offer_price', 'type_of_security',
       'description', 'marketplace',
       'primary_exchange_where_issue_will_be_listed', 'HFILE', 'LFILE',
       'original_middle_of_filing_price_range', 'shares_filed_in_this_mkt',
       'amt_filed_in_this_mkt_($_mil)', 'reit_type_code', 'depos_itary',
       'unit_inv._trust', 'closed_end_fund/trust_flag_(y/n)',
       'gross_spread_as_%_of_prncpl_amt_in_this_mkt', 'AH_LFILE', 'AH_HFILE',
       'change_in_value_of_shares

In [ ]:
CCM = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/code2/data/CCM2.csv')
CCM['sic3'] = CCM['sic'].astype(str).str[:3]
CCM

,GVKEY,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,conm,tic,cusip,sic,ipodate,FYRC,sic3
0,1000,LU,25881,23369,11/13/1970,6/30/1978,A & E PLASTIK PAK INC,AE.2,32102,3089.0,NaN,12,308
1,1001,LU,10015,6398,9/20/1983,7/31/1986,A & M FOOD SERVICES INC,AMFD.,165100,5812.0,NaN,12,581
2,1002,LC,10023,22159,12/14/1972,6/5/1973,AAI CORP,AAIC.1,352104,3825.0,NaN,12,382
3,1003,LU,10031,6672,12/7/1983,8/16/1989,A.A. IMPORTING CO INC,ANTQ,354100,5712.0,NaN,1,571
4,1004,LU,54594,20000,4/24/1972,E,AAR CORP,AIR,361105,5080.0,1/1/1988,5,508
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32708,349972,LC,15642,55395,8/4/2021,E,INDAPTUS THERAPEUTICS INC,INDP,45339J105,2836.0,8/4/2021,12,283
32709,349994,LC,23514,59438,11/15/2022,E,CLEARMIND MEDICINE INC,CMND,185053204,2836.0,NaN,10,283
32710,350681,LC,22205,58855,10/22/2021,E,GETNET ADQUIRENCIA E,GET,37428A103,7374.0,10/18/2021,12,737
32711,351038,LC,16161,55612,10/29/2021,E,QUOIN PHARMACEUTICALS LTD,QNRX,74907L300,2834.0,NaN,12,283


In [ ]:
CCM[CCM.duplicated(subset=['GVKEY','LPERMNO'], keep=False)].shape

In [ ]:
crsp_1995_2022.shape

(40917860, 10)

In [ ]:
# Merging 'crsp_monthly' and 'CCM' on 'permno'
crsp_ccm_1 = pd.merge(crsp_1995_2022, CCM[['LPERMNO', 'GVKEY','LINKDT','LINKENDDT']], how="inner", left_on='PERMNO', right_on='LPERMNO')
crsp_ccm_1

,PERMNO,date,SHRCD,EXCHCD,NCUSIP,PRC,RET,SHROUT,OPENPRC,NCUSIP6,LPERMNO,GVKEY,LINKDT,LINKENDDT
0,10001,1995-01-03,11,3,29274A10,8.375,0.046875,2224,8.500,29274A,10001,12994,1/9/1986,8/31/2017
1,10001,1995-01-04,11,3,29274A10,8.000,-0.044776,2224,8.000,29274A,10001,12994,1/9/1986,8/31/2017
2,10001,1995-01-05,11,3,29274A10,7.500,-0.062500,2224,8.000,29274A,10001,12994,1/9/1986,8/31/2017
3,10001,1995-01-06,11,3,29274A10,-7.750,0.033333,2224,NaN,29274A,10001,12994,1/9/1986,8/31/2017
4,10001,1995-01-09,11,3,29274A10,7.500,-0.032258,2224,7.500,29274A,10001,12994,1/9/1986,8/31/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47227950,93436,2022-12-23,11,3,88160R10,123.150,-0.017551,3157752,126.370,88160R,93436,184996,6/29/2010,E
47227951,93436,2022-12-27,11,3,88160R10,109.100,-0.114089,3157752,117.495,88160R,93436,184996,6/29/2010,E
47227952,93436,2022-12-28,11,3,88160R10,112.710,0.033089,3157752,110.350,88160R,93436,184996,6/29/2010,E
47227953,93436,2022-12-29,11,3,88160R10,121.820,0.080827,3157752,120.385,88160R,93436,184996,6/29/2010,E


In [ ]:
crsp_ccm_1[crsp_ccm_1.duplicated(subset=['PERMNO','date'], keep=False)].shape

(3057240, 14)

In [ ]:
crsp_ccm_1.to_csv('/content/sample_data/crsp_ccm_1.csv')

In [ ]:
# change the LINKDT date formate
crsp_ccm_1['LINKDT'] = [ datetime.strptime(d,'%m/%d/%Y').date() for d in tqdm(crsp_ccm_1['LINKDT'])]

# change the LINKENDDT formate to compare the dates
link_end_date=crsp_ccm_1.LINKENDDT.copy()
for i,e in enumerate(tqdm(crsp_ccm_1.LINKENDDT)):
  if e=='E':
    link_end_date[i] = datetime.max.date()
  else:
    link_end_date[i] = datetime.strptime(e, '%m/%d/%Y').date()
crsp_ccm_1['LINKENDDT'] = link_end_date

100%|██████████| 47227955/47227955 [12:22<00:00, 63581.89it/s]


In [ ]:
# change the LINKDT date formate
crsp_ccm_1['date'] = [ datetime.strptime(d,'%Y-%m-%d').date() for d in tqdm(crsp_ccm_1['date'])]

100%|██████████| 47227955/47227955 [05:51<00:00, 134412.98it/s]


In [ ]:
# remove the dates that are not applicable
crsp_ccm_2 = crsp_ccm_1[(crsp_ccm_1['date']>=crsp_ccm_1['LINKDT']) & (crsp_ccm_1['date']<=crsp_ccm_1['LINKENDDT'])]
crsp_ccm_2.shape

(40765345, 14)

In [ ]:
crsp_ccm_2

In [ ]:
crsp_ccm_2.drop(['LPERMNO','LINKDT','LINKENDDT'],axis=1, inplace=True)

<ipython-input-12-9e2eb8229c1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_ccm_2.drop(['LPERMNO','LINKDT','LINKENDDT'],axis=1, inplace=True)


In [ ]:
# market capitalization in million
crsp_ccm_2['mkt_cap_million'] = (crsp_ccm_2.SHROUT * crsp_ccm_2.PRC)/1000

<ipython-input-13-60c9d9bd8f43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_ccm_2['mkt_cap_million'] = (crsp_ccm_2.SHROUT * crsp_ccm_2.PRC)/1000


In [ ]:
crsp_ccm_2['date'].iloc[0].year

1995

In [ ]:
# make year variable
crsp_ccm_2['fyear'] = [fyear.year for fyear in crsp_ccm_2['date']]

<ipython-input-27-be15013f61f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_ccm_2['fyear'] = [fyear.year for fyear in crsp_ccm_2['date']]


In [ ]:
crsp_ccm_2

,PERMNO,date,SHRCD,EXCHCD,NCUSIP,PRC,RET,SHROUT,OPENPRC,NCUSIP6,GVKEY,mkt_cap_million,fyear
0,10001,1995-01-03,11,3,29274A10,8.375,0.046875,2224,8.500,29274A,12994,18.62600,1995
1,10001,1995-01-04,11,3,29274A10,8.000,-0.044776,2224,8.000,29274A,12994,17.79200,1995
2,10001,1995-01-05,11,3,29274A10,7.500,-0.062500,2224,8.000,29274A,12994,16.68000,1995
3,10001,1995-01-06,11,3,29274A10,-7.750,0.033333,2224,NaN,29274A,12994,-17.23600,1995
4,10001,1995-01-09,11,3,29274A10,7.500,-0.032258,2224,7.500,29274A,12994,16.68000,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47227950,93436,2022-12-23,11,3,88160R10,123.150,-0.017551,3157752,126.370,88160R,184996,388877.15880,2022
47227951,93436,2022-12-27,11,3,88160R10,109.100,-0.114089,3157752,117.495,88160R,184996,344510.74320,2022
47227952,93436,2022-12-28,11,3,88160R10,112.710,0.033089,3157752,110.350,88160R,184996,355910.22792,2022
47227953,93436,2022-12-29,11,3,88160R10,121.820,0.080827,3157752,120.385,88160R,184996,384677.34864,2022


In [ ]:
# compustat sic code data
sic_1995_2022 =  pd.read_csv('https://utexas.box.com/shared/static/kpqim4mj7j4a174khqfxrfo4yan3qax8.csv')
sic_1995_2022

In [ ]:
sic_1995_2022[sic_1995_2022.duplicated(subset=['gvkey','fyear','sic'], keep=False)].shape

(65885, 14)

In [ ]:
# fyear
sic_1995_2022.fyear = sic_1995_2022.fyear.astype(str).str[:4]
# fyear
sic_1995_2022['sic3'] = sic_1995_2022.sic.astype(str).str[:3]

In [ ]:
sic_1995_2022.rename(columns={"gvkey":"GVKEY"}, inplace=True)
sic_1995_2022.rename(columns={"datadate":"date"}, inplace=True)

<ipython-input-43-dcf64273a02b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sic_1995_2022.rename(columns={"gvkey":"GVKEY"}, inplace=True)
<ipython-input-43-dcf64273a02b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sic_1995_2022.rename(columns={"datadate":"date"}, inplace=True)


In [ ]:
fyear_list = []
for d in tqdm(sic_1995_2022['fyear']):
  if d!='nan':
    fyear_list.append(datetime.strptime(d,'%Y').year)
  else:
    fyear_list.append(None)

100%|██████████| 325182/325182 [00:02<00:00, 155955.42it/s]


In [ ]:
sic_1995_2022['fyear'] = fyear_list
sic_1995_2022

In [ ]:
# drop duplicates
sic_1995_2022 = sic_1995_2022.drop_duplicates(subset=['GVKEY','datadate','sic3'], keep='first')

In [ ]:
sic_1995_2022

,GVKEY,date,fyear,indfmt,consol,popsrc,datafmt,cusip,conm,curcd,act,costat,mkvalt,sic,sic3
0,1004,1995-05-31,1994,INDL,C,D,STD,000361105,AAR CORP,USD,321.632,A,NaN,5080,508
1,1004,1996-05-31,1995,INDL,C,D,STD,000361105,AAR CORP,USD,338.012,A,NaN,5080,508
2,1004,1997-05-31,1996,INDL,C,D,STD,000361105,AAR CORP,USD,414.100,A,NaN,5080,508
3,1004,1998-05-31,1997,INDL,C,D,STD,000361105,AAR CORP,USD,468.400,A,NaN,5080,508
4,1004,1999-05-31,1998,INDL,C,D,STD,000361105,AAR CORP,USD,508.186,A,540.7748,5080,508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358061,352262,2022-12-31,2022,INDL,C,D,STD,G2415A113,COOL COMPANY LTD,USD,145.338,A,NaN,4400,440
358062,353444,2021-12-31,2021,INDL,C,D,STD,405552100,HALEON PLC,USD,7110.379,A,NaN,2834,283
358063,353444,2022-12-31,2022,INDL,C,D,STD,405552100,HALEON PLC,USD,4890.689,A,NaN,2834,283
358064,353945,2021-12-31,2021,INDL,C,D,STD,00449R109,ACCELLERON INDUSTRIES AG,USD,436.863,A,NaN,3621,362


In [ ]:
# merge
crsp_sic_merge_2 = pd.merge(crsp_ccm_2, sic_1995_2022[['GVKEY','fyear','act','sic3']], on=['GVKEY','fyear'], how='left')
crsp_sic_merge_2

,PERMNO,date,SHRCD,EXCHCD,NCUSIP,PRC,RET,SHROUT,OPENPRC,NCUSIP6,GVKEY,mkt_cap_million,fyear,act,sic3
0,10001,1995-01-03,11,3,29274A10,8.375,0.046875,2224,8.500,29274A,12994,18.62600,1995,6.263,492
1,10001,1995-01-04,11,3,29274A10,8.000,-0.044776,2224,8.000,29274A,12994,17.79200,1995,6.263,492
2,10001,1995-01-05,11,3,29274A10,7.500,-0.062500,2224,8.000,29274A,12994,16.68000,1995,6.263,492
3,10001,1995-01-06,11,3,29274A10,-7.750,0.033333,2224,NaN,29274A,12994,-17.23600,1995,6.263,492
4,10001,1995-01-09,11,3,29274A10,7.500,-0.032258,2224,7.500,29274A,12994,16.68000,1995,6.263,492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40765340,93436,2022-12-23,11,3,88160R10,123.150,-0.017551,3157752,126.370,88160R,184996,388877.15880,2022,40917.000,371
40765341,93436,2022-12-27,11,3,88160R10,109.100,-0.114089,3157752,117.495,88160R,184996,344510.74320,2022,40917.000,371
40765342,93436,2022-12-28,11,3,88160R10,112.710,0.033089,3157752,110.350,88160R,184996,355910.22792,2022,40917.000,371
40765343,93436,2022-12-29,11,3,88160R10,121.820,0.080827,3157752,120.385,88160R,184996,384677.34864,2022,40917.000,371


In [ ]:
# no duplicates
crsp_sic_merge_2[crsp_sic_merge_2.duplicated(subset=['PERMNO','date'], keep=False)].shape

(0, 15)

In [ ]:
# no negative price error
crsp_sic_merge_2.mkt_cap_million = crsp_sic_merge_2.mkt_cap_million.abs( )
crsp_sic_merge_2.PRC = crsp_sic_merge_2.PRC.abs( )

In [ ]:
crsp_sic_merge_2.drop(['SHRCD','EXCHCD','NCUSIP6','fyear'],axis=1, inplace=True)

In [ ]:
crsp_sic_merge_2.to_csv('/content/drive/MyDrive/UT_research/VC_IB/code2/data/crsp_sic_merge_2.csv')

In [ ]:
crsp_sic_merge_2

,PERMNO,date,NCUSIP,PRC,RET,SHROUT,OPENPRC,GVKEY,mkt_cap_million,act,sic3
0,10001,1995-01-03,29274A10,8.375,0.046875,2224,8.500,12994,18.62600,6.263,492
1,10001,1995-01-04,29274A10,8.000,-0.044776,2224,8.000,12994,17.79200,6.263,492
2,10001,1995-01-05,29274A10,7.500,-0.062500,2224,8.000,12994,16.68000,6.263,492
3,10001,1995-01-06,29274A10,7.750,0.033333,2224,NaN,12994,17.23600,6.263,492
4,10001,1995-01-09,29274A10,7.500,-0.032258,2224,7.500,12994,16.68000,6.263,492
...,...,...,...,...,...,...,...,...,...,...,...
40765340,93436,2022-12-23,88160R10,123.150,-0.017551,3157752,126.370,184996,388877.15880,40917.000,371
40765341,93436,2022-12-27,88160R10,109.100,-0.114089,3157752,117.495,184996,344510.74320,40917.000,371
40765342,93436,2022-12-28,88160R10,112.710,0.033089,3157752,110.350,184996,355910.22792,40917.000,371
40765343,93436,2022-12-29,88160R10,121.820,0.080827,3157752,120.385,184996,384677.34864,40917.000,371


## New Section

In [ ]:
# unique permno list
CRSP_permno_list = list(crsp_sic_merge_2.PERMNO.unique())

In [ ]:
# sort values
crsp_ccm_2 = crsp_ccm_2.sort_values(['PERMNO', 'date'])

In [ ]:
crsp_1995_2022_dict = dict()
for permno in tqdm(CRSP_permno_list):
  crsp_1995_2022_dict[permno] = crsp_sic_merge_2[crsp_sic_merge_2.PERMNO == permno ]

In [ ]:
crsp_1995_2022_dict[permno]

In [ ]:
# save dictionary to crsp_1995_2022_dict.pkl file
with open('/content/drive/MyDrive/UT_research/VC_IB/code2/data/crsp_1995_2022_dict.pkl', 'wb') as fp:
    pickle.dump(crsp_1995_2022_dict, fp)